# Restricted Boltzman Machine 

A restricted Boltzmann Machine is an "Energy Based" generative stochastic model. They were initially invented by Paul Smolensky in 1986 and were called "Harmonium". After the evolution of training algorithms in the mid 2000's by Geoffrey Hinton, the boltzman machine became more prominent. They gained big popularity in recent years in the context of the Netflix Prize where RBMs achieved state of the art performance in collaborative filtering and have beaten most of the competition.

RBM's are useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling. 


## Architecture
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. As we move forward we would learn why the output layer won't be needed.
<img src="images/figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="images/figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [56]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state

In [57]:
# graded
# import data

df = pd.read_excel("input/amazon.xlsx")

In [58]:
#run this and check if you have got the correct output
df.head()

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, you find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. You train your RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, let's convert the train and test data you split above into a bag of words model.

In [88]:
# graded

# create bag of words model on train and test data
tf = CountVectorizer(stop_words='english') #the final shape should be (number of documents, vocabulary)

# fit tf on the dataframe df
tf = tf.fit(df['Text'])

# transform df dataframe
trainX = tf.transform(df['Text'])

In [89]:
#check if you are getting the correct output
print(sum(trainX.toarray()[1]))
trainX.shape

# there are thousand documents and 1642 words in the dictionary 
# removed stop words other there are 1847

4


(1000, 1642)

You should get:
<br>

3 <br>
(1000, 1825)

Now that you have the bag of words model, let's define the number of visible and hidden units.

In [90]:
# graded

# define visible units
visibleUnits = trainX.shape[1] # vocabulary size ~1 line

# assign number of units
hiddenUnits = 5 # hyperparameter, this means that we are looking for 5 topics 

visibleUnits # visibleunits are words in the dictionary. these are the nodes in visible layer. 

1642

In [62]:
# graded
# utility Functions

# deine the sigmoid function
def sigmoid(X):
    return 1/(1+np.exp(-X)) # ~ 1 line X here energy function (v,h)

**Joint Probability**  
Probability of both variables occurring together. In this case probability of a given word vector belonging to particular class. 

**Conditional Probability**  
Probability of a particular word vector (document) given a class. 
Probability of a particular class given a particular word vector (document). 


## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="images/figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

*MyNotes: What is energy function. As explained it talks about chemistry between two variables. If they are related energy will be low else high. So E(v,h) will be low if given a value v h can co-exist or be a related value*

*MyNotes in this case v is vocabulary (1642 words) for 1000 documents and h are 5 categories. Summation here is over all 1000 documents for each category.*

[MyNotes] **Joint probability** here means for a given set of word vector, which category the document belongs. Probability of document belonging to one of the 5 (hyperparameter) category.

The **joint distribution** is known as the **Boltzmann Distribution** which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and **conditional probabilities** of state v given the state h:
<img src="images/figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="images/figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

Let's look at *sampleHiddenLayer* now.

### Sample Hidden Layer

You already know that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="images/figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

<font color=grey>
    i,j notation means i = input variables = 1632 in V layer and j is output that is 5 in our case 
    for practice you need to add sample size. for vi will be matric of sample size * 1632 (number of neurons) 
    wij is weights across all i so will be matric of 1632 * 1 (*number of samples) so it will 1632*n. ViWij output should be of size sample size. and b is also same size so should be able to add.
</font>

In [180]:
# graded
# here weights should be passed for a specific jth hidden layer or hiddenbias should be for all j (hidden neurons)
def sampleHiddenLayer(v_sample):  #sample = 200*1642 (no of docs*features)
#     print('inside sampleHiddenLayer')
    # write the code for calculation of hPdf using vectorized implementation of Eq 4
#     print('hiddenBias.shape, W.shape, v_sample.shape ', hiddenBias.shape, v_sample.shape, W.shape ) 
    # sizes hiddenBias: batchsize*5 W: v*h (1642*5) v_sample: batchsize*1642(v) : number of documents 
    hPdf = sigmoid(hiddenBias + np.dot(v_sample, W)) # ~ 1 line
#     print('hPdf.shape: ', hPdf.shape)
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
#     print('h_sample.shape: ', h_sample.shape)
#     print('end of sampleHiddenLayer')
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="images/figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, you are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
   

 <font color=grey>
    Wij for a given i will be of size 5*n (sample size) 
    hj (will of size 5*1)
    visibleBias is of size 1632
 </font>

In [186]:
#graded
def sampleVisibleLayer(h_sample, D):
#     print('start: sampleVisibleLayer')
    # complete the following function such that vPdf has the sum of entries equal to 1 for each of the datapoints in the batch
    # you have to use axis = 1 in writing the denominator
#     print(visibleBias.shape, h_sample.shape, W.shape)
    numerator = sigmoid(visibleBias + np.dot(h_sample,np.transpose(W))) # ~1 line output is 1642*200 
    
#     print('numerator.shape: ', numerator.shape) # batchsize * visiblenodes 
    
    denominator = numerator.sum(axis=1) # ~1 line  output is 1642
#     print('denominator.shape: ', denominator.shape)
    
    # vPdf: visible node prob distribution. for each document (row), 1642 columns will store their probability
    vPdf = np.transpose(numerator)/denominator # ~1 line
    vPdf = np.transpose(vPdf)
#     print('vPdf.shape: ', vPdf.shape)
    
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((batchSize, vPdf.shape[1]))
#     print('v_sample.shape: ', v_sample.shape)
    
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
#     print('end: sampleVisibleLayer')
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. Note that we are calculating the visible layer samples first and then using it to calculate he hidden layer sample. It'll become clear soon why we are doing so when you write the function for Contrastive Divergence.
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [182]:
#graded
def gibbs(h_sample, D):
#     print('start gibbs')
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample, D)[0], sampleVisibleLayer(h_sample, D)[1] # ~1 line
    hPdf, h_sample = sampleHiddenLayer(v_sample) # ~1 line
#     print('end gibbs')
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

You have learned that Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [183]:
#graded
def cd_k(data,k): # data here is noofdocs*1632 (features in doc). batchsize is 200 then its 200*1632 shuffled documents
#     print('start cd_k')
    D = data.sum(axis=1) # sums by each row. for each doc. so D.shape = 1,200
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data) # sample the hidden layer using the input data
    chain_start = hiddenSample_data
    
    for step in range(k):
#         print('step: ', step)
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start, D) # perform gibbs sampling using chain_start
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples, D) # perform gibbs sampling using hiddenSamples
    return hiddenPDF_data, visibleSamples, hiddenPDF
#     print('end cd_k')

## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="images/figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. **Figure out** the vectorized implementation for this.

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>
These will be helpful in writing the weight updates.

In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

In [136]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 100
eta = 0.05 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)  # this in our case of size 1642*5

visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

print('W.shape: ', W.shape)
print('visibleBias.shape: ', visibleBias.shape)
print('hiddenBias.shape: ', hiddenBias.shape)

# we shall use 10 interations of gibbs sampling
k=10

W.shape:  (1642, 5)
visibleBias.shape:  (1642,)
hiddenBias.shape:  (5,)


### Training

In [195]:
#graded
def train(dataX,k): # dataX is number of documents by features 1000*1632 datapoints here 
    
    mW = np.zeros((visibleUnits, hiddenUnits)) # initialise momentum_weights Wij (i=visibleunits, j=hiddenunits)
    mvisibleBias = np.zeros(visibleUnits) # initialise momentum_visiblebiases
    mhiddenBias =   np.zeros(hiddenUnits) # initialise momentum_hiddenbiases
    
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs # calling the variables initiliazed at the begining
    
    hiddenBias = np.zeros((batchSize,hiddenUnits))
    visibleBias = np.zeros((batchSize,visibleUnits))
    for epoch in range(epochs): # 100 epocs 
        np_rng.shuffle(dataX) # shuffling the data
        print('epoch: ', epoch)
        
        for i in range(0, dataX.shape[0], batchSize): # batchSize 200. data.shape[0] = 1000 docs. so it will run 5 times once for 200 datapoints
            mData = dataX[i:i+batchSize] # select a batch of datapoints .. Its already shuffled 
    
            hiddenPDF_data, visibleSamples, hiddenPDF = cd_k(mData,k) # perfrom Contrastive Divergence on the batch for k iterations
            
            #mW = v*h  visibleSampe[0]=1*v hidden=200*h 
            mW = mrate*mW - (np.outer(visibleSamples[0], hiddenPDF_data[0])-np.outer(visibleSamples[-1], hiddenPDF[-1])) # write the momentum update equation for weight matrix

            mvisibleBias =  mrate*mvisibleBias - np.mean(visibleSamples[0]-visibleSamples[-1]) #write the momentum update equation for visiblebias vector
            mhiddenBias =  mrate*mhiddenBias - np.mean(hiddenPDF_data - hiddenPDF) #write the momentum update equation for hiddenbias vector
            
            W = W + eta*mW #weight update equation
            visibleBias = visibleBias + eta*mvisibleBias  #visible bias update equation
            hiddenBias =  hiddenBias + eta*mhiddenBias #hidden bias update equation  



## Train the Model

This will take around 10 minutes of time to run.

In [196]:
train(trainX.toarray(),k)

epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17
epoch:  18
epoch:  19
epoch:  20
epoch:  21
epoch:  22
epoch:  23
epoch:  24
epoch:  25
epoch:  26
epoch:  27
epoch:  28
epoch:  29
epoch:  30
epoch:  31
epoch:  32
epoch:  33
epoch:  34
epoch:  35
epoch:  36
epoch:  37
epoch:  38
epoch:  39
epoch:  40
epoch:  41
epoch:  42
epoch:  43
epoch:  44
epoch:  45
epoch:  46
epoch:  47
epoch:  48
epoch:  49
epoch:  50
epoch:  51
epoch:  52
epoch:  53
epoch:  54
epoch:  55
epoch:  56
epoch:  57
epoch:  58
epoch:  59
epoch:  60
epoch:  61
epoch:  62
epoch:  63
epoch:  64
epoch:  65
epoch:  66
epoch:  67
epoch:  68
epoch:  69
epoch:  70
epoch:  71
epoch:  72
epoch:  73
epoch:  74
epoch:  75
epoch:  76
epoch:  77
epoch:  78
epoch:  79
epoch:  80
epoch:  81
epoch:  82
epoch:  83
epoch:  84
epoch:  85
epoch:  86
epoch:  87
epoch:  88
epoch:  89
epoch:  90
epoch:  9

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. You can take a look at the words under each topic and see what they are talking about. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. You can see that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary
    
Feel free to change the number of iterations of gibbs sampling in Contrastive Divergence and see how the distribution of words change under the topics.

In [197]:
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias)
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten()
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            lm+=1
            if lm==15:
                break
        print('\n')

In [198]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
promptly 4.7575324311605434e-06
cutouts 4.6119748585615525e-06
warning 4.597179697156521e-06
year 4.548676296856357e-06
ripped 4.45585296558716e-06
allow 4.292373015398564e-06
certainly 4.2591010558396355e-06
drops 4.1834321440386825e-06
700w 4.1709009954123565e-06
sucked 4.090435087006367e-06
13 4.0711921403692004e-06
menus 4.059201912689368e-06
management 4.048250409268541e-06
things 4.030587067404241e-06
reboots 4.000715912971854e-06


topic 1 : [0. 1. 0. 0. 0.]
promptly 4.805985438762818e-06
warning 4.712003641827225e-06
cutouts 4.657256581337736e-06
year 4.440727172411431e-06
ripped 4.4327957599617565e-06
700w 4.21031092836661e-06
allow 4.2077281310574806e-06
certainly 4.157211600573346e-06
just 4.142659240069382e-06
management 4.1124307501483135e-06
13 4.110792472932081e-06
drops 4.041201525960416e-06
menus 4.037583518119325e-06
fact 4.0350120404406545e-06
fee 4.0247300206772814e-06


topic 2 : [0. 0. 1. 0. 0.]
promptly 4.787119877193233e-06
warning 4.6

In [199]:
tf.vocabulary_

{'way': 1582,
 'plug': 1081,
 'unless': 1524,
 'converter': 326,
 'good': 639,
 'case': 218,
 'excellent': 508,
 'value': 1552,
 'great': 645,
 'jawbone': 774,
 'tied': 1458,
 'charger': 240,
 'conversations': 325,
 'lasting': 809,
 '45': 20,
 'minutes': 921,
 'major': 882,
 'problems': 1114,
 'mic': 914,
 'jiggle': 776,
 'line': 835,
 'right': 1227,
 'decent': 373,
 'volume': 1562,
 'dozen': 438,
 'contacts': 316,
 'imagine': 720,
 'fun': 608,
 'sending': 1277,
 'razr': 1161,
 'owner': 1016,
 'needless': 954,
 'say': 1256,
 'wasted': 1579,
 'money': 929,
 'waste': 1578,
 'time': 1460,
 'sound': 1358,
 'quality': 1146,
 'impressed': 724,
 'going': 637,
 'original': 1003,
 'battery': 128,
 'extended': 526,
 'seperated': 1281,
 'mere': 906,
 'ft': 605,
 'started': 1378,
 'notice': 972,
 'excessive': 511,
 'static': 1383,
 'garbled': 618,
 'headset': 675,
 'design': 385,
 'odd': 980,
 'ear': 457,
 'clip': 266,
 'comfortable': 278,
 'highly': 685,
 'recommend': 1187,
 'blue': 153,
 'tooth'

The output shows the probability assigned to each word for ever topic present.